<a href="https://colab.research.google.com/github/dmswl0707/transfer-learning_pytorch/blob/main/training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
def train(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

best_model_wts=copy.deepcopy(model.state_dic())
best_acc=0.0

for epoch in range(num_epochs):
    print('Epoch {}/{}'.format(epoch, num_epochs - 1))
    print('_'*10)

    #each epoch 
    for phase in ['train','val']:
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-'*10)
    
    for phase in ['train','val']:
        if phase == 'train':
            schedular.step()
            model.train(True)
        else:
            model.train(False)
    
    running_loss=